<a href="https://colab.research.google.com/github/paul-kim88/pythonProject2/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!pip install lxml[html_clean] readability-lxml pdfminer.six

import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html
from io import BytesIO
from pdfminer.high_level import extract_text

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"#&iar=news&ia=news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, "html.parser")

        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})

        return results
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# def extract(text):
#     text = re.sub(r'<.*?>', '', text)
#     text = re.sub(r'[\t\n]+', '\n', text)
#     text = html.escape(text)
#     return text

def extract(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text = re.sub(r'[\t\n]+', '\n', text)
    text = html.escape(text)
    return text

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    search_results = duckduckgo_scrape(search_query, '3d')

    results = []

    if search_results:
        print(f"Search Results for '{search_query}':")

        for result in search_results:
            if 'wiki' in result['link']:
                continue
            print(f"Title: {result['title']}")
            print(f"Link: {result['link']}")
            print("-" * 20)

            try:
                response = requests.get(result['link'])
                response.raise_for_status()

                if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
                    pdf_file = BytesIO(response.content)
                    content = extract_text(pdf_file)
                else:
                    soup = BeautifulSoup(response.content, "html.parser")
                    doc = Document(response.content)
                    content = extract(doc.summary())

                results.append({"title": result['title'], "link": result['link'], "content": content})

            except requests.exceptions.HTTPError as e:
                print(f"HTTP Error fetching {result['link']}: {e}")
            except Exception as e:
                print(f"Error fetching or processing {result['link']}: {e}")


        # for result in search_results:
        #     if 'wiki' in result['link']:
        #       continue
        #     print(f"Title: {result['title']}")
        #     print(f"Link: {result['link']}")
        #     print("-" * 20)

        #     # Try to get the link, handling HTTP errors
        #     try:
        #         response = requests.get(result['link'])
        #         response.raise_for_status()  # Raise an exception for bad status codes

        #         soup = BeautifulSoup(response.content, "html.parser")
        #         doc = Document(response.content)
        #         content = extract(doc.summary())
        #         results.append({"title": result['title'], "link": result['link'], "content": content})

        #     except requests.exceptions.HTTPError as e:
        #         print(f"HTTP Error fetching {result['link']}: {e}")
        #     except Exception as e:
        #         print(f"Error fetching or processing {result['link']}: {e}")
    search_df=pd.DataFrame(results)
    search_df

Type What you want : LG Chem sustainability
Search Results for 'LG Chem sustainability':
Title: LG Chem Sustainability Business
Link: https://www.lgchem-sustainability.com/
--------------------
Title: Sustainability Strategy - LG Chem
Link: https://www.lgchem.com/sustainability/sustainability-strategy?lang=en_GLOBAL
--------------------
Title: Sustainability Report and ESG Performance Data | Sustainability ...
Link: https://www.lgchem.com/sustainability/sustainability-strategy/esg-data?lang=en_US
--------------------
Title: Sustainability Report and ESG Performance Data | Sustainability ...
Link: https://www.lgchem.com/sustainability/sustainability-strategy/sustainability-reports?lang=en_US
--------------------
Title: Press Release | Media - LG
Link: https://www.lgcorp.com/media/release/22110
--------------------
Title: Press Release | Media - LG
Link: https://lgcorp.com/media/release/23649
--------------------
Title: LG Chem Ltd 2020 Sustainability Report - SustainabilityReports.com
L

In [9]:
df = pd.DataFrame(results)
df


,title,link,content
0,LG Chem Sustainability Business,https://www.lgchem-sustainability.com/,Compostable materials are bioplastics that\nde...
1,Sustainability Strategy - LG Chem,https://www.lgchem.com/sustainability/sustaina...,\nThe page you requested could not be found.\n...
2,Sustainability Report and ESG Performance Data...,https://www.lgchem.com/sustainability/sustaina...,\nTax Strategy\n ...
3,Sustainability Report and ESG Performance Data...,https://www.lgchem.com/sustainability/sustaina...,\nSustainability Report and ESG Performance Da...
4,Press Release | Media - LG,https://www.lgcorp.com/media/release/22110,&quot;Will Reduce 1-year Carbon Emissions from...
5,Press Release | Media - LG,https://lgcorp.com/media/release/23649,\nLG Chem\nLG Chem Publishes Sustainable Manag...
6,LG Chem Ltd 2020 Sustainability Report - Susta...,https://sustainabilityreports.com/reports/lg-c...,\nThe report highlights LG Chem&#x27;s commitm...
7,LG Chem Blog,https://blog.lgchem.com/en/2023/08/24_sustaina...,\nThe Earth is on the brink of unprecedented c...
8,LG Chem Ltd 2021 Sustainability Report,https://sustainabilityreports.com/reports/lg-c...,\nThe report outlines LG Chem&#x27;s commitmen...
9,LG Chem Ltd 2019 Sustainability Report,https://sustainabilityreports.com/reports/lg-c...,\nThe report details LG Chem&#x27;s sustainabi...


In [27]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html
from io import BytesIO
from pdfminer.high_level import extract_text

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
        if len(content) < 100:
            return None
        return {"title": result['title'], "link": result['link'], "content": content}

    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        print(f"Search Results for '{search_query}':")
        with requests.Session() as session:
            for result in search_results:
                if 'wiki' in result['link']:
                    continue
                print(f"Title: {result['title']}")
                print(f"Link: {result['link']}")
                print("-" * 20)
                processed = process_link(result, session)
                if processed:
                    results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : LG화학 친환경소재
Search Results for 'LG화학 친환경소재':
Title: 세계 최초 생분해성 신소재를 개발한 Lg화학! 친환경 소재를 개발해 자원 선순환에 앞장섭니다
Link: https://blog.lgchem.com/2020/11/16_biodegradable-material/
--------------------
Title: LG화학 - LG Chem
Link: https://www.lgchem.com/main/index?lang=ko_KR
--------------------
Title: 생분해성 플라스틱의 핵심 원료 3hp! Lg화학이 세계 최초로 상용화에 도전합니다
Link: https://blog.lgchem.com/2023/11/16_3hp/
--------------------
Title: 친환경 바이오 밸런스드(Bio-balanced) 제품으로 국내 최초 친환경 국제인증을 받은 LG화학! LG화학이 개발한 식물성 ...
Link: https://blog.lgchem.com/2021/07/12_bio_balanced/
--------------------
Title: Lg화학, 충남 대산에 친환경 소재 공장 10개 짓는다… 2.6조원 투자
Link: https://biz.chosun.com/industry/company/2021/08/19/7MKWN7EXOVHIPK2SMVXRNKTBC4/
--------------------
Title: 플라스틱 대체할 '생분해성 신소재' 세계 최초 개발
Link: https://www.chosun.com/economy/industry-company/2021/09/30/JKAUSYFTJFDQ7F7OAVLCRQ7R5Q/
--------------------
Title: ①불에 잘 안 타면서 ②재활용 플라스틱까지 더한 친환경 난연 소재 개발한 Lg화학 | 한국일보
Link: https://www.hankookilbo.com/News/Read/A202408121

In [21]:
search_df

,title,link,content
0,"故 김새론, 마지막 포착된 모습 보니…""혼자 있을 때 어두워 보였다""",https://www.dailian.co.kr/news/view2/1462454,"ⓒ영화 '기타맨' 스틸컷\n배우 故 김새론이 16일 사망한 가운데, 그의 유작에도 ..."
1,배우 김새론 사망… SNS 추모 물결 이어져 - news-wa.com,https://news-wa.com/article/entertainment/cele...,얼마전 SNS 프로필서 故 문빈 추모로 안타까움 더해\n김새론 / 사진 = 김새론 ...
2,[속보] 배우 김새론 서울 자택에서 숨진 채 발견 - 매일경제,https://www.mk.co.kr/news/society/11242263,사진 확대\n배우 김새론. [사진 출처 = 김새론 인스타그램]\n배우 김새론(25)...
3,"김새론 사망 외신 반응 ""구하라·설리 이어 한국 연예산업의 비극""",https://www.news-wa.com/article/entertainment/...,사진=김새론 SNS\n배우 김새론(25)의 갑작스러운 사망 소식이 전해지면서 국내외...
4,"천재 소녀 김새론, 안타까운 사망 [이슈in] - MBC NEWS",https://imnews.imbc.com/news/2025/enter/articl...,영화 '아저씨'에서 주연을 맡으며 일약 스타덤에 오른 배우 김새론(25)이 세상을 ...
5,2025 김새론 배우 근황 + 작품활동 + 최신소식,https://newneek.co/@user_yiz8zggr8s/article/17947,[2025 김새론 배우 근황 + 작품활동 + 최신소식]\n아역배우 출신 배우 김새론...
6,배우 김새론 프로필 나이 사망 - azzibang.com,https://azzibang.com/entry/%EB%B0%B0%EC%9A%B0-...,카테고리 없음\n배우 김새론 프로필 나이 사망\nby 우리 아빠는 자영업자\n202...
7,"김새론 사망, 프로필,사건 및 사고, 작품활동 총정리",https://lei-ben.com/entry/%EA%B9%80%EC%83%88%E...,"카테고리 없음\n김새론 사망, 프로필,사건 및 사고, 작품활동 총정리\nby 레이벤..."
8,"'아저씨'로 주목받았던 배우 김새론, 16일 사망…음주운전 자숙→복귀 앞둔 근황 어땠나",https://www.topstarnews.net/news/articleView.h...,(톱스타뉴스 이수현 기자) 배우 김새론(본명 김아임)이 갑작스럽게 세상을 떠났다.\...


In [31]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html
from io import BytesIO
from pdfminer.high_level import extract_text
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
            if len(content) < 100:
                return None  # 내용이 100자 미만이면 건너뜁니다.
        return {"title": result['title'], "link": result['link'], "content": content}
    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        print(f"Search Results for '{search_query}':")
        with requests.Session() as session:
          with ThreadPoolExecutor(max_workers=10) as executor:
              future_to_result = {}
              for result in search_results:
                  if 'wiki' in result['link']:
                      continue
                  print(f"Title: {result['title']}")
                  print(f"Link: {result['link']}")
                  print("-" * 20)

                  future = executor.submit(process_link, result, session)
                  future_to_result[future] = result

              for future in as_completed(future_to_result):
                  processed = future.result()
                  if processed:
                      results.append(processed)
            # with ThreadPoolExecutor(max_workers=20) as executor:
            #     future_to_result = {
            #         executor.submit(process_link, result, session): result
            #         for result in search_results
            #         if 'wiki' not in result['link']
            #     }
            #     for future in as_completed(future_to_result):
            #         processed = future.result()
            #         if processed:
            #             results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : llama3 파인튜닝
Search Results for 'llama3 파인튜닝':
Title: [우아한 스터디] LLM(LLaMA3) Fine-Tuning 방법 정리 - 벨로그
Link: https://velog.io/@judy_choi/LLMLLaMA3-Fine-Tuning-%EB%B0%A9%EB%B2%95-%EC%A0%95%EB%A6%AC
--------------------
Title: [Python] Llama3를 파인튜닝을 통해 나만의 데이터로 학습 및 Huggingface에 적재해보자.
Link: https://unfinishedgod.netlify.app/2024/05/24/python/
--------------------
Title: FSDP (Fully Shared Data Parallel) : Llama3 70B 모델을 멀티 GPU로 파인튜닝 방법 ...
Link: https://m.blog.naver.com/se2n/223451837382
--------------------
Title: llama 3.1 간단하게 Fine Tuning 하기 — 나의 표류일지
Link: https://zzzini-5.tistory.com/315
--------------------
Title: [Llama3.1] Ollama 로 Fine tuning 하기 - Good coder
Link: https://bob-data.tistory.com/41
--------------------
Title: Llama-3.2-1B-Instruction 모델 파인튜닝하기 — NJSUNG BLOG
Link: https://naakjii.tistory.com/138
--------------------
Title: 메타 라마3(Llama3) 8B 한글 요약 모델 파인튜닝 구현과 구글 Gemma 7B 한글 요약 성능 비교 검토
Link: https://m.blog.naver.com/se2n/223421081812
---------------

In [32]:
search_df

,title,link,content
0,[Python] Llama3를 파인튜닝을 통해 나만의 데이터로 학습 및 Huggin...,https://unfinishedgod.netlify.app/2024/05/24/p...,파인튜닝을 통해 나만의 데이터를 학습 시켜 보자. 그전에 파인튜닝에 대해 알아 봐야...
1,메타 라마3(Llama3) 8B 한글 요약 모델 파인튜닝 구현과 구글 Gemma 7...,https://m.blog.naver.com/se2n/223421081812,"​\n비씨카드가 신규 회원수 1위에 올랐고, 하위권 카드사가 잘 만든 상품 하나로 ..."
2,FSDP (Fully Shared Data Parallel) : Llama3 70B...,https://m.blog.naver.com/se2n/223451837382,​\n* 첨언 : FSDP vs DDP (Distributed Data Parall...
3,llama 3.1 간단하게 Fine Tuning 하기 — 나의 표류일지,https://zzzini-5.tistory.com/315,Unsloth를 참고하여 llama 3.1을 파인튜닝하는 과정을 정리해보았다.\n(...
4,Llama-3.2-1B-Instruction 모델 파인튜닝하기 — NJSUNG BLOG,https://naakjii.tistory.com/138,"Llama 모델이란?\n먼저, Llama 모델에 대해 간단히 소개하겠습니다.\nDa..."
5,[Llama3.1] Ollama 로 Fine tuning 하기 - Good coder,https://bob-data.tistory.com/41,Llama를 한 번쯤은 로컬에서 구동해보고 싶었는데 이번에 Llama3.1 이 나오...
